### Importing libraries

In [1]:
import requests
from urllib import request
from bs4 import BeautifulSoup
import time
import pandas as pd
import datetime as dt
import os
import re

In [2]:
### ignores copy warning
pd.options.mode.chained_assignment = None 

### Downloading precinct-level report and setting dataframe

In [3]:
### creates directory to download report from website
os.makedirs("senate_dade/reports_raw/", exist_ok = True)
### creates directory to store final dataframe after exporting
os.makedirs("senate_dade/report_files/", exist_ok = True)

### sets URL
url = "https://enr.electionsfl.org/DAD/3267/Reports/"

### retrieves data
page = requests.get(url)
### converts data to text for reading
soup = BeautifulSoup(page.text)

### stores all a-tags with links in them
a_report = soup.find_all("a", href = True)

### retrieves the download link
file_link = a_report[-2].get("href")
### stores time of access
tnow = dt.datetime.now()
### constructs file name
filename = "raw_report_" + str(tnow.strftime("%m-%d-%Y-%H%M")) + ".csv"

### parses data
r = requests.get(file_link)

### checks if file is available to download, if not it prints an error message
if r.status_code != 200:
    print(f"Error downloading {file_link}")
### downloads file
else:
    with open("senate_dade/reports_raw/" + filename, "wb") as outfile:
        outfile.write(r.content)

### imports the file as dataframe
df_raw = pd.read_csv("senate_dade/reports_raw/" + filename)

### displays dataframe
df_raw

,Contest,Candidate Issue,Party,Precinct Name,Mail Votes,Early Votes,Election Day Votes,Total Votes,% of Total Votes
0,United States Senator,Marco Rubio,REP,PRECINCT 001.0,118,111,70,299,69.37
1,United States Senator,Marco Rubio,REP,PRECINCT 010.0,-,-,-,32,91.43
2,United States Senator,Marco Rubio,REP,PRECINCT 100.0,189,0,0,189,26.00
3,United States Senator,Marco Rubio,REP,PRECINCT 100.2,43,0,0,43,14.93
4,United States Senator,Marco Rubio,REP,PRECINCT 100.3,64,0,0,64,15.35
...,...,...,...,...,...,...,...,...,...
50671,SMiami Installation Elected Official,No,NaN,PRECINCT 651.0,-,-,-,73,23.47
50672,SMiami Installation Elected Official,No,NaN,PRECINCT 652.0,-,-,-,70,20.90
50673,SMiami Installation Elected Official,No,NaN,PRECINCT 653.0,-,-,-,63,14.79
50674,SMiami Installation Elected Official,No,NaN,PRECINCT 653.1,-,-,-,4,15.38


### Filtering on relevant election-type

In [4]:
for i in range(0, len(df_raw)):
    df_raw["Contest"][i] = str(df_raw["Contest"][i].upper())

### filters for specific contest
df_filter = df_raw[df_raw["Contest"] == "UNITED STATES SENATOR"].reset_index(drop = True)

### loop runs through filtered dataframe
for i in range(0, len(df_filter)):
    ### converts party to string type (null values were being read as float)
    df_filter["Party"][i] = str(df_filter["Party"][i])
    ### replaces with "OTHER"
    df_filter["Party"][i] = df_filter["Party"][i].replace("nan", "OTHER")
    df_filter["Candidate Issue"][i] = df_filter["Candidate Issue"][i].replace("WRITE-IN", "Write-In")

### displays dataframe
df_filter

,Contest,Candidate Issue,Party,Precinct Name,Mail Votes,Early Votes,Election Day Votes,Total Votes,% of Total Votes
0,UNITED STATES SENATOR,Marco Rubio,REP,PRECINCT 001.0,118,111,70,299,69.37
1,UNITED STATES SENATOR,Marco Rubio,REP,PRECINCT 010.0,-,-,-,32,91.43
2,UNITED STATES SENATOR,Marco Rubio,REP,PRECINCT 100.0,189,0,0,189,26.00
3,UNITED STATES SENATOR,Marco Rubio,REP,PRECINCT 100.2,43,0,0,43,14.93
4,UNITED STATES SENATOR,Marco Rubio,REP,PRECINCT 100.3,64,0,0,64,15.35
...,...,...,...,...,...,...,...,...,...
6313,UNITED STATES SENATOR,Write-In,OTHER,PRECINCT 999.0,0,0,0,0,0.00
6314,UNITED STATES SENATOR,Write-In,OTHER,PRECINCT 999.1,0,0,0,0,0.00
6315,UNITED STATES SENATOR,Write-In,OTHER,PRECINCT 999.2,0,0,0,0,0.00
6316,UNITED STATES SENATOR,Write-In,OTHER,PRECINCT 999.3,0,0,0,0,0.00


### Extracting base precincts and compiling results into precinct-level

Note: Comment out this block if compilation is not needed. Add a command saying the following:
df_voteprecinct = df_filter.copy()

In [5]:
### creates new columns to be written into
df_filter["Precinct"] = " "
df_filter["Base_Precinct"] = " "

### loop runs through dataframe
for i in range(0, len(df_filter)):
    ### extracts precinct number and stores as a list
    precinct = re.findall('\d*\.?\d+', df_filter["Precinct Name"][i])
    ### extracts precinct number from list (in this case list has only one element) and writes into cell
    df_filter["Precinct"][i] = precinct[0]
    ### extracts base precinct number and writes into cell
    df_filter["Base_Precinct"][i] = int(df_filter["Precinct"][i][:3])

### displays dataframe
df_filter

,Contest,Candidate Issue,Party,Precinct Name,Mail Votes,Early Votes,Election Day Votes,Total Votes,% of Total Votes,Precinct,Base_Precinct
0,UNITED STATES SENATOR,Marco Rubio,REP,PRECINCT 001.0,118,111,70,299,69.37,001.0,1
1,UNITED STATES SENATOR,Marco Rubio,REP,PRECINCT 010.0,-,-,-,32,91.43,010.0,10
2,UNITED STATES SENATOR,Marco Rubio,REP,PRECINCT 100.0,189,0,0,189,26.00,100.0,100
3,UNITED STATES SENATOR,Marco Rubio,REP,PRECINCT 100.2,43,0,0,43,14.93,100.2,100
4,UNITED STATES SENATOR,Marco Rubio,REP,PRECINCT 100.3,64,0,0,64,15.35,100.3,100
...,...,...,...,...,...,...,...,...,...,...,...
6313,UNITED STATES SENATOR,Write-In,OTHER,PRECINCT 999.0,0,0,0,0,0.00,999.0,999
6314,UNITED STATES SENATOR,Write-In,OTHER,PRECINCT 999.1,0,0,0,0,0.00,999.1,999
6315,UNITED STATES SENATOR,Write-In,OTHER,PRECINCT 999.2,0,0,0,0,0.00,999.2,999
6316,UNITED STATES SENATOR,Write-In,OTHER,PRECINCT 999.3,0,0,0,0,0.00,999.3,999


In [6]:
### converts number of votes into integers
df_filter["Total Votes"] = df_filter["Total Votes"].astype(int)
### groups sub-precincts into base precincts
df_voteprecinct = df_filter.groupby(["Base_Precinct", "Candidate Issue", "Party"]).agg({"Total Votes": "sum"}).reset_index()

### loop runs through newly created dataframe
for i in range(0, len(df_voteprecinct)):
    ### converts base precinct numbers into strings
    df_voteprecinct["Base_Precinct"][i] = str(df_voteprecinct["Base_Precinct"][i])

### displays dataframe
df_voteprecinct

,Base_Precinct,Candidate Issue,Party,Total Votes
0,1,Dennis Misigoy,LPF,1
1,1,Marco Rubio,REP,299
2,1,Steven B. Grant,NPA,1
3,1,Tuan TQ Nguyen,NPA,0
4,1,Val Demings,DEM,130
...,...,...,...,...
4693,999,Marco Rubio,REP,102
4694,999,Steven B. Grant,NPA,0
4695,999,Tuan TQ Nguyen,NPA,0
4696,999,Val Demings,DEM,165


### Converting datasets into horizontal format

In [7]:
### stores each precinct number
precincts = df_voteprecinct["Base_Precinct"].explode().unique() 

In [8]:
### stores names of each candidate
candidates = df_voteprecinct["Candidate Issue"].explode().unique() 
### computes and stores number of candidates
candidate_len = len(candidates)

In [9]:
### initializes new list (referred to as "final list" later) to write data into
final_rows = []

### loop runs through dataframe
for i in range(0, len(precincts)):
    
    ### stores time of processing/update
    tnow = dt.datetime.now()
    dt_string = tnow.strftime("%m-%d-%Y %H:%M")
    update = "Last updated at " + str(dt_string)
    
    ### creates new list with first element as processing time
    pre_row = [update]
    
    ### stores base precinct number
    precinct = precincts[i]
    
    ### adds base precinct number to list
    pre_row.append(precinct)
    
    ### nested loop runs through dataframe 
    for j in range(0, len(df_voteprecinct)):
        
        ### if the base precinct number matches, then candidate details are stored in list
        if df_voteprecinct["Base_Precinct"][j] == precinct:
            
            ### adds party to the candidate name in format <NAME (PARTY)>
            can_name = df_voteprecinct["Candidate Issue"][j] + " (" + str(df_voteprecinct["Party"][j]) + ")" 
            
            ### adds candidate name to list
            pre_row.append(can_name)
            ### adds votes won to list
            pre_row.append(df_voteprecinct["Total Votes"][j])
            ### adds party name to list
            pre_row.append(df_voteprecinct["Party"][j])
            
            ### adds list to final list
            final_rows.append(pre_row)
        else:
            continue
    
    ### converts final list to dataframe
    df_initial = pd.DataFrame(final_rows)

### displays dataframe
df_initial

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,Last updated at 02-14-2023 03:16,1,Dennis Misigoy (LPF),1,LPF,Marco Rubio (REP),299,REP,Steven B. Grant (NPA),1,NPA,Tuan TQ Nguyen (NPA),0,NPA,Val Demings (DEM),130,DEM,Write-In (OTHER),0,OTHER
1,Last updated at 02-14-2023 03:16,1,Dennis Misigoy (LPF),1,LPF,Marco Rubio (REP),299,REP,Steven B. Grant (NPA),1,NPA,Tuan TQ Nguyen (NPA),0,NPA,Val Demings (DEM),130,DEM,Write-In (OTHER),0,OTHER
2,Last updated at 02-14-2023 03:16,1,Dennis Misigoy (LPF),1,LPF,Marco Rubio (REP),299,REP,Steven B. Grant (NPA),1,NPA,Tuan TQ Nguyen (NPA),0,NPA,Val Demings (DEM),130,DEM,Write-In (OTHER),0,OTHER
3,Last updated at 02-14-2023 03:16,1,Dennis Misigoy (LPF),1,LPF,Marco Rubio (REP),299,REP,Steven B. Grant (NPA),1,NPA,Tuan TQ Nguyen (NPA),0,NPA,Val Demings (DEM),130,DEM,Write-In (OTHER),0,OTHER
4,Last updated at 02-14-2023 03:16,1,Dennis Misigoy (LPF),1,LPF,Marco Rubio (REP),299,REP,Steven B. Grant (NPA),1,NPA,Tuan TQ Nguyen (NPA),0,NPA,Val Demings (DEM),130,DEM,Write-In (OTHER),0,OTHER
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4693,Last updated at 02-14-2023 03:17,999,Dennis Misigoy (LPF),0,LPF,Marco Rubio (REP),102,REP,Steven B. Grant (NPA),0,NPA,Tuan TQ Nguyen (NPA),0,NPA,Val Demings (DEM),165,DEM,Write-In (OTHER),0,OTHER
4694,Last updated at 02-14-2023 03:17,999,Dennis Misigoy (LPF),0,LPF,Marco Rubio (REP),102,REP,Steven B. Grant (NPA),0,NPA,Tuan TQ Nguyen (NPA),0,NPA,Val Demings (DEM),165,DEM,Write-In (OTHER),0,OTHER
4695,Last updated at 02-14-2023 03:17,999,Dennis Misigoy (LPF),0,LPF,Marco Rubio (REP),102,REP,Steven B. Grant (NPA),0,NPA,Tuan TQ Nguyen (NPA),0,NPA,Val Demings (DEM),165,DEM,Write-In (OTHER),0,OTHER
4696,Last updated at 02-14-2023 03:17,999,Dennis Misigoy (LPF),0,LPF,Marco Rubio (REP),102,REP,Steven B. Grant (NPA),0,NPA,Tuan TQ Nguyen (NPA),0,NPA,Val Demings (DEM),165,DEM,Write-In (OTHER),0,OTHER


In [10]:
### removes duplicates
df = df_initial[::candidate_len].reset_index(drop = True)

### displays dataframe
df

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,Last updated at 02-14-2023 03:16,1,Dennis Misigoy (LPF),1,LPF,Marco Rubio (REP),299,REP,Steven B. Grant (NPA),1,NPA,Tuan TQ Nguyen (NPA),0,NPA,Val Demings (DEM),130,DEM,Write-In (OTHER),0,OTHER
1,Last updated at 02-14-2023 03:16,2,Dennis Misigoy (LPF),3,LPF,Marco Rubio (REP),594,REP,Steven B. Grant (NPA),6,NPA,Tuan TQ Nguyen (NPA),1,NPA,Val Demings (DEM),247,DEM,Write-In (OTHER),0,OTHER
2,Last updated at 02-14-2023 03:16,3,Dennis Misigoy (LPF),5,LPF,Marco Rubio (REP),1024,REP,Steven B. Grant (NPA),11,NPA,Tuan TQ Nguyen (NPA),5,NPA,Val Demings (DEM),351,DEM,Write-In (OTHER),0,OTHER
3,Last updated at 02-14-2023 03:16,4,Dennis Misigoy (LPF),10,LPF,Marco Rubio (REP),679,REP,Steven B. Grant (NPA),5,NPA,Tuan TQ Nguyen (NPA),6,NPA,Val Demings (DEM),353,DEM,Write-In (OTHER),1,OTHER
4,Last updated at 02-14-2023 03:16,5,Dennis Misigoy (LPF),6,LPF,Marco Rubio (REP),1063,REP,Steven B. Grant (NPA),8,NPA,Tuan TQ Nguyen (NPA),2,NPA,Val Demings (DEM),573,DEM,Write-In (OTHER),0,OTHER
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
778,Last updated at 02-14-2023 03:17,994,Dennis Misigoy (LPF),4,LPF,Marco Rubio (REP),289,REP,Steven B. Grant (NPA),0,NPA,Tuan TQ Nguyen (NPA),4,NPA,Val Demings (DEM),149,DEM,Write-In (OTHER),0,OTHER
779,Last updated at 02-14-2023 03:17,995,Dennis Misigoy (LPF),6,LPF,Marco Rubio (REP),968,REP,Steven B. Grant (NPA),3,NPA,Tuan TQ Nguyen (NPA),3,NPA,Val Demings (DEM),1084,DEM,Write-In (OTHER),0,OTHER
780,Last updated at 02-14-2023 03:17,996,Dennis Misigoy (LPF),2,LPF,Marco Rubio (REP),456,REP,Steven B. Grant (NPA),1,NPA,Tuan TQ Nguyen (NPA),1,NPA,Val Demings (DEM),698,DEM,Write-In (OTHER),0,OTHER
781,Last updated at 02-14-2023 03:17,997,Dennis Misigoy (LPF),1,LPF,Marco Rubio (REP),135,REP,Steven B. Grant (NPA),0,NPA,Tuan TQ Nguyen (NPA),0,NPA,Val Demings (DEM),52,DEM,Write-In (OTHER),0,OTHER


### Computing total votes tallied

In [11]:
### storing number of columns
col_no = len(df.columns)

### creates new column to write totals into at the end
### since this is an index-number, it automatically computes the required column number irrespective of number of candidates
df[col_no] = " "

### creates list to store tallied votes
votes_tallied = []

### loop runs through newly created horizontal dataframe
for i in range(0, len(df)):
    
    ### initializes variable to store total
    total = 0
    
    ### nested loop starts at column 3 (that is the first column with a vote number)
    ### loop iterates (number of candidates*3 + 1) at intervals of three to read all tallied votes
    ### note: each candidate has three corresponding elements—name,votes,party—hence the (n*3+1)
    for j in range(3, (candidate_len*3 + 1), 3):
        ### adds votes to total
        total = total + df[j][i]
    
    ### adds total to list
    votes_tallied.append(total)
    ### writes total into new column
    df[col_no][i] = total

### displays dataframe
df

,0,1,2,3,4,5,6,7,8,9,...,11,12,13,14,15,16,17,18,19,20
0,Last updated at 02-14-2023 03:16,1,Dennis Misigoy (LPF),1,LPF,Marco Rubio (REP),299,REP,Steven B. Grant (NPA),1,...,Tuan TQ Nguyen (NPA),0,NPA,Val Demings (DEM),130,DEM,Write-In (OTHER),0,OTHER,431
1,Last updated at 02-14-2023 03:16,2,Dennis Misigoy (LPF),3,LPF,Marco Rubio (REP),594,REP,Steven B. Grant (NPA),6,...,Tuan TQ Nguyen (NPA),1,NPA,Val Demings (DEM),247,DEM,Write-In (OTHER),0,OTHER,851
2,Last updated at 02-14-2023 03:16,3,Dennis Misigoy (LPF),5,LPF,Marco Rubio (REP),1024,REP,Steven B. Grant (NPA),11,...,Tuan TQ Nguyen (NPA),5,NPA,Val Demings (DEM),351,DEM,Write-In (OTHER),0,OTHER,1396
3,Last updated at 02-14-2023 03:16,4,Dennis Misigoy (LPF),10,LPF,Marco Rubio (REP),679,REP,Steven B. Grant (NPA),5,...,Tuan TQ Nguyen (NPA),6,NPA,Val Demings (DEM),353,DEM,Write-In (OTHER),1,OTHER,1054
4,Last updated at 02-14-2023 03:16,5,Dennis Misigoy (LPF),6,LPF,Marco Rubio (REP),1063,REP,Steven B. Grant (NPA),8,...,Tuan TQ Nguyen (NPA),2,NPA,Val Demings (DEM),573,DEM,Write-In (OTHER),0,OTHER,1652
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
778,Last updated at 02-14-2023 03:17,994,Dennis Misigoy (LPF),4,LPF,Marco Rubio (REP),289,REP,Steven B. Grant (NPA),0,...,Tuan TQ Nguyen (NPA),4,NPA,Val Demings (DEM),149,DEM,Write-In (OTHER),0,OTHER,446
779,Last updated at 02-14-2023 03:17,995,Dennis Misigoy (LPF),6,LPF,Marco Rubio (REP),968,REP,Steven B. Grant (NPA),3,...,Tuan TQ Nguyen (NPA),3,NPA,Val Demings (DEM),1084,DEM,Write-In (OTHER),0,OTHER,2064
780,Last updated at 02-14-2023 03:17,996,Dennis Misigoy (LPF),2,LPF,Marco Rubio (REP),456,REP,Steven B. Grant (NPA),1,...,Tuan TQ Nguyen (NPA),1,NPA,Val Demings (DEM),698,DEM,Write-In (OTHER),0,OTHER,1158
781,Last updated at 02-14-2023 03:17,997,Dennis Misigoy (LPF),1,LPF,Marco Rubio (REP),135,REP,Steven B. Grant (NPA),0,...,Tuan TQ Nguyen (NPA),0,NPA,Val Demings (DEM),52,DEM,Write-In (OTHER),0,OTHER,188


### Computing precinct-level vote shares, leads and margins

In [12]:
### creates new list to store index numbers of newly created columns
new_cols = []

### loop runs through number of candidates and four more for other details
for i in range(0, (candidate_len + 4)):   
    ### creates (n+4) new columns for "n" candidates
    df[col_no + i + 1] = " "
    ### stores numbers in a list as strings
    new_cols.append(str(col_no + i + 1))

### loop runs through each row of dataframe
for i in range(0, len(df)):
    
    ### creates lists to store details
    votes_obtained = []
    party = []
    can_names = []
    
    ### stores the total votes tallied for each row
    total = votes_tallied[i]
    
    if total != 0: 
    
        ## TALLYING VOTES

        ### nested loop starts at column 3 (that is the first column with a vote number)
        ### loop iterates (number of candidates*3 + 1) at intervals of three to read all details
        ### note: each candidate has three corresponding elements—name,votes,party—hence the (n*3+1)
        for j in range(3, (candidate_len*3 + 1), 3):

            ### stores votes, party and candidate names in the relevant lists
            ### note: here format is [cols x rows] while traversing through dataframe
            votes_obtained.append(df[j][i])
            party.append(df[j+1][i])
            can_names.append(df[j-1][i])


        ## COMPUTING VOTE-SHARES

        ### loop runs through all the candidates
        for k in range(0, candidate_len):
            ### column-index of a candidate is given by corresponding element in "new_cols"
            ### first candidate (k = 0) will be stored in column at index-0 in the "new_cols" list and so on
            column_no = int(new_cols[k])
            ### computes percent and writes into the column
            df[column_no][i] = (votes_obtained[k]/total*100).round(2)

        ## COMPUTING LEADING CANDIDATES AND WIN MARGINS

        ### computes and stores the highest vote-tally for the "votes_obtained" list
        max_value = max(votes_obtained)

        ### if no votes have been tallied
        #if max_value == 0:

            #lead_party = "N/A"
            #lead_can = "No votes tallied."
            #margin = 0
            #margin_pp = 0

        ### checks if there are more elements with same value (in case of ties)
        if votes_obtained.count(max_value) > 1:

            ### method returns indices of all duplicate highest vote-tallies
            indices = []
            def find_indices(list_to_check, item_to_find):
                return [idx for idx, value in enumerate(list_to_check) if value == item_to_find]

            ### stores indices of all duplicate highest vote-tallies
            indices = find_indices(votes_obtained, max_value)

            ### constructs string of all candidate names corresponding to highest vote-tallies
            string = []
            for ind in range(0, len(indices)):
                index = indices[ind]
                string.append(can_names[index])

            ### stores value as "TIED"
            lead_party = "TIED"
            ### stores candidate names with tied votes
            lead_can = " and ".join([", ".join(string[:-1]),string[-1]] if len(string) > 2 else string)
            ### stores margin of difference as 0
            margin = 0
            margin_pp = 0

        else:
            ### computes and stores the index at which the highest vote is stored in the "votes_obtained" list
            max_index = votes_obtained.index(max_value)
            ### stores name of party corresponding to highest vote-tally in the fourth-last column
            ### if highest vote-tally is second (index=1) element in "votes_obtained"
            ### the second element (index=1) in "party" list is the corresponding datapoint
            lead_party = party[max_index]
            ### similarly, stores name of candidate corresponding to highest vote-tally in the next (third-last) column
            lead_can = can_names[max_index]

            ### computes and stores the second highest vote-tally for the "votes_obtained" list
            second_max_value = max(votes_obtained, key = lambda x: min(votes_obtained)-1 if (x == max_value) else x)
            ### computes and stores the raw number of votes between first and second placed candidates
            margin = max_value - second_max_value
            ### computes the difference in percent points between first and second placed candidates
            margin_pp = ((max_value/total*100)-(second_max_value/total*100)).round(2)

        ### stores index-number of the fourth-last element of "new_cols" list (this will be the fourth-last column)
        lead_col = int(new_cols[-4])
        ### stores name of party corresponding to highest vote-tally in the fourth-last column
        ### if highest vote-tally is second (index=1) element in "votes_obtained"
        ### the second element (index=1) in "party" list is the corresponding datapoint
        df[lead_col][i] = lead_party
        ### similarly, stores name of candidate corresponding to highest vote-tally in the next (third-last) column
        df[lead_col+1][i] = lead_can

        ### stores index-number of the penultimate element of "new_cols" list (this will be the penultimate column)
        raw_margin_col = int(new_cols[-2])
        ### stores raw margin of difference in penultimate column
        df[raw_margin_col][i] = margin

        ### stores index-number of the last element of "new_cols" list (this will be the last column)
        margin_pp_col = int(new_cols[-1])
        ### stores percent points difference in last column
        df[margin_pp_col][i] = margin_pp
    
    else: 
        
        ### loop runs through all candidates
        for n in range(0, candidate_len):
            ### column-index of a candidate is given by corresponding element in "new_cols"
            ### first candidate (k = 0) will be stored in column at index-0 in the "new_cols" list and so on
            column_no = int(new_cols[n])
            ### computes percent and writes into the column
            df[column_no][i] = 0
        
        lead_party = "No votes tallied"
        lead_can = "No votes tallied"
        margin = 0
        margin_pp = 0
        
        ### stores index-number of the fourth-last element of "new_cols" list (this will be the fourth-last column)
        lead_col = int(new_cols[-4])
        ### stores name of party corresponding to highest vote-tally in the fourth-last column
        ### if highest vote-tally is second (index=1) element in "votes_obtained"
        ### the second element (index=1) in "party" list is the corresponding datapoint
        df[lead_col][i] = lead_party
        ### similarly, stores name of candidate corresponding to highest vote-tally in the next (third-last) column
        df[lead_col+1][i] = lead_can

        ### stores index-number of the penultimate element of "new_cols" list (this will be the penultimate column)
        raw_margin_col = int(new_cols[-2])
        ### stores raw margin of difference in penultimate column
        df[raw_margin_col][i] = margin

        ### stores index-number of the last element of "new_cols" list (this will be the last column)
        margin_pp_col = int(new_cols[-1])
        ### stores percent points difference in last column
        df[margin_pp_col][i] = margin_pp       

### replaces null values with "0"
df.fillna(0, inplace = True)    
   
### displays dataframe
df

,0,1,2,3,4,5,6,7,8,9,...,21,22,23,24,25,26,27,28,29,30
0,Last updated at 02-14-2023 03:16,1,Dennis Misigoy (LPF),1,LPF,Marco Rubio (REP),299,REP,Steven B. Grant (NPA),1,...,0.23,69.37,0.23,0.00,30.16,0.00,REP,Marco Rubio (REP),169,39.21
1,Last updated at 02-14-2023 03:16,2,Dennis Misigoy (LPF),3,LPF,Marco Rubio (REP),594,REP,Steven B. Grant (NPA),6,...,0.35,69.80,0.71,0.12,29.02,0.00,REP,Marco Rubio (REP),347,40.78
2,Last updated at 02-14-2023 03:16,3,Dennis Misigoy (LPF),5,LPF,Marco Rubio (REP),1024,REP,Steven B. Grant (NPA),11,...,0.36,73.35,0.79,0.36,25.14,0.00,REP,Marco Rubio (REP),673,48.21
3,Last updated at 02-14-2023 03:16,4,Dennis Misigoy (LPF),10,LPF,Marco Rubio (REP),679,REP,Steven B. Grant (NPA),5,...,0.95,64.42,0.47,0.57,33.49,0.09,REP,Marco Rubio (REP),326,30.93
4,Last updated at 02-14-2023 03:16,5,Dennis Misigoy (LPF),6,LPF,Marco Rubio (REP),1063,REP,Steven B. Grant (NPA),8,...,0.36,64.35,0.48,0.12,34.69,0.00,REP,Marco Rubio (REP),490,29.66
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
778,Last updated at 02-14-2023 03:17,994,Dennis Misigoy (LPF),4,LPF,Marco Rubio (REP),289,REP,Steven B. Grant (NPA),0,...,0.90,64.80,0.00,0.90,33.41,0.00,REP,Marco Rubio (REP),140,31.39
779,Last updated at 02-14-2023 03:17,995,Dennis Misigoy (LPF),6,LPF,Marco Rubio (REP),968,REP,Steven B. Grant (NPA),3,...,0.29,46.90,0.15,0.15,52.52,0.00,DEM,Val Demings (DEM),116,5.62
780,Last updated at 02-14-2023 03:17,996,Dennis Misigoy (LPF),2,LPF,Marco Rubio (REP),456,REP,Steven B. Grant (NPA),1,...,0.17,39.38,0.09,0.09,60.28,0.00,DEM,Val Demings (DEM),242,20.90
781,Last updated at 02-14-2023 03:17,997,Dennis Misigoy (LPF),1,LPF,Marco Rubio (REP),135,REP,Steven B. Grant (NPA),0,...,0.53,71.81,0.00,0.00,27.66,0.00,REP,Marco Rubio (REP),83,44.15


### Computing overall county vote shares, leads and margins

In [13]:
## COMPUTING COUNTY-LEVEL TOTALS

### storing number of columns
new_col_no = len(df.columns)

total_col = candidate_len*3 + 2
df[new_col_no] = df[total_col].sum() 
ct_total = df[total_col].sum()

### creates new list to store index numbers of newly created columns
ct_cols = []

### stores new length of dataframe
ct_new_col_no = len(df.columns)

if ct_total != 0:

    ## COMPUTING COUNTY-LEVEL VOTE SHARES

    ### computes sum of all votes per candidate
    ct_can_votes = []
    for j in range(3, (candidate_len*3 + 1), 3):
        ct_can_votes.append(df[j].sum())

    ### computes vote-shares of candidates
    for i in range(0, candidate_len):
        df[ct_new_col_no + i] = (ct_can_votes[i]/ct_total*100).round(2)

    ## COMPUTING LEADING CANDIDATES AND WIN MARGINS

    ### computes new length of dataframe
    final_col_no = len(df.columns)

    ### stores overall maximum votes
    ct_max_value = max(ct_can_votes)

    ### if no votes have been tallied
    #if ct_max_value == 0:

        #ct_lead_party = "N/A"
        #ct_lead_can = "No votes tallied."
        #ct_margin = 0
        #ct_margin_pp = 0

    ### if there are more than one instances of highest votes (in case of a tie)
    if ct_can_votes.count(ct_max_value) > 1:

        ### method returns indices of all duplicate highest vote-tallies
        indices = []
        def find_indices(list_to_check, item_to_find):
            return [idx for idx, value in enumerate(list_to_check) if value == item_to_find]

        ### stores indices of all duplicate highest vote-tallies
        indices = find_indices(ct_can_votes, ct_max_value)

        ### constructs string of all candidate names corresponding to highest votes tallied
        string = []
        for ind in range(0, len(indices)):
            index = indices[ind]
            string.append(can_names[index])

        ### stores leading party as tied
        ct_lead_party = "TIED"
        ### stores names of tied candidates
        ct_lead_can = " and ".join([", ".join(string[:-1]),string[-1]] if len(string) > 2 else string)

        ### stores margins as "0"
        ct_margin = 0
        ct_margin_pp = 0

        ### stores vote share for candidate with highest vote-tally
        ct_lead_voteshare = (ct_max_value/ct_total*100).round(2)

    else: 

        ### computes index of the highest tallied vote
        ct_max_index = ct_can_votes.index(ct_max_value)

        ### stores name of corresponding party and candidates
        ct_lead_party = party[ct_max_index]
        ct_lead_can = can_names[ct_max_index]

        ### computes second highest tallied vote
        second_ct_max_value = max(ct_can_votes, key = lambda x: min(ct_can_votes)-1 if (x == ct_max_value) else x)
        ### computes index of the second-highest tallied vote
        second_ct_max_index = ct_can_votes.index(second_ct_max_value)
        
        ### stores name of corresponding party and candidates
        second_ct_party = party[second_ct_max_index]
        second_ct_can = can_names[second_ct_max_index]

        ### stores margin of difference and the margin as percent points
        ct_margin = ct_max_value - second_ct_max_value
        ct_margin_pp = ((ct_max_value/ct_total*100)-(second_ct_max_value/ct_total*100)).round(2)

        ### stores the vote share of the highest tallied vote
        ct_lead_voteshare = (ct_max_value/ct_total*100).round(2)
        ### stores the vote share of the second highest tallied vote
        second_ct_voteshare = (second_ct_max_value/ct_total*100).round(2)

    ### writes into cells
    df[final_col_no + 1] = ct_lead_party
    df[final_col_no + 2] = ct_lead_can
    df[final_col_no + 3] = ct_margin
    df[final_col_no + 4] = ct_margin_pp
    df[final_col_no + 5] = ct_lead_voteshare
    df[final_col_no + 6] = second_ct_party
    df[final_col_no + 7] = second_ct_can
    df[final_col_no + 8] = second_ct_voteshare
    
else: 
    
    ### sets vote-shares of candidates as 0
    for i in range(0, candidate_len):
        df[ct_new_col_no + i] = 0
        
    ### computes new length of dataframe
    final_col_no = len(df.columns)
    
    ct_lead_party = "No votes tallied"
    ct_lead_can = "No votes tallied"
    ct_margin = 0
    ct_margin_pp = 0
    ct_lead_voteshare = 0
    second_ct_party = "No votes tallied"
    second_ct_can = "No votes tallied"
    second_ct_voteshare = 0
    
    ### writes into cells
    df[final_col_no + 1] = ct_lead_party
    df[final_col_no + 2] = ct_lead_can
    df[final_col_no + 3] = ct_margin
    df[final_col_no + 4] = ct_margin_pp
    df[final_col_no + 5] = ct_lead_voteshare
    df[final_col_no + 6] = second_ct_party
    df[final_col_no + 7] = second_ct_can
    df[final_col_no + 8] = second_ct_voteshare

### replaces null values with "0"
df.fillna(0, inplace = True)    
   
### displays dataframe
df

,0,1,2,3,4,5,6,7,8,9,...,36,37,39,40,41,42,43,44,45,46
0,Last updated at 02-14-2023 03:16,1,Dennis Misigoy (LPF),1,LPF,Marco Rubio (REP),299,REP,Steven B. Grant (NPA),1,...,44.8,0.04,REP,Marco Rubio (REP),67273,9.45,54.25,DEM,Val Demings (DEM),44.8
1,Last updated at 02-14-2023 03:16,2,Dennis Misigoy (LPF),3,LPF,Marco Rubio (REP),594,REP,Steven B. Grant (NPA),6,...,44.8,0.04,REP,Marco Rubio (REP),67273,9.45,54.25,DEM,Val Demings (DEM),44.8
2,Last updated at 02-14-2023 03:16,3,Dennis Misigoy (LPF),5,LPF,Marco Rubio (REP),1024,REP,Steven B. Grant (NPA),11,...,44.8,0.04,REP,Marco Rubio (REP),67273,9.45,54.25,DEM,Val Demings (DEM),44.8
3,Last updated at 02-14-2023 03:16,4,Dennis Misigoy (LPF),10,LPF,Marco Rubio (REP),679,REP,Steven B. Grant (NPA),5,...,44.8,0.04,REP,Marco Rubio (REP),67273,9.45,54.25,DEM,Val Demings (DEM),44.8
4,Last updated at 02-14-2023 03:16,5,Dennis Misigoy (LPF),6,LPF,Marco Rubio (REP),1063,REP,Steven B. Grant (NPA),8,...,44.8,0.04,REP,Marco Rubio (REP),67273,9.45,54.25,DEM,Val Demings (DEM),44.8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
778,Last updated at 02-14-2023 03:17,994,Dennis Misigoy (LPF),4,LPF,Marco Rubio (REP),289,REP,Steven B. Grant (NPA),0,...,44.8,0.04,REP,Marco Rubio (REP),67273,9.45,54.25,DEM,Val Demings (DEM),44.8
779,Last updated at 02-14-2023 03:17,995,Dennis Misigoy (LPF),6,LPF,Marco Rubio (REP),968,REP,Steven B. Grant (NPA),3,...,44.8,0.04,REP,Marco Rubio (REP),67273,9.45,54.25,DEM,Val Demings (DEM),44.8
780,Last updated at 02-14-2023 03:17,996,Dennis Misigoy (LPF),2,LPF,Marco Rubio (REP),456,REP,Steven B. Grant (NPA),1,...,44.8,0.04,REP,Marco Rubio (REP),67273,9.45,54.25,DEM,Val Demings (DEM),44.8
781,Last updated at 02-14-2023 03:17,997,Dennis Misigoy (LPF),1,LPF,Marco Rubio (REP),135,REP,Steven B. Grant (NPA),0,...,44.8,0.04,REP,Marco Rubio (REP),67273,9.45,54.25,DEM,Val Demings (DEM),44.8


### Computing and inserting column names

In [14]:
### creates list to store column names with two initial elements
cols = ["LAST_UPDATE_2022", "BASE_PRECINCT"]

### loop runs through all the candidates
for i in range(0, candidate_len):
    ### generates column names for each candidate and adds to list
    cols.append("CANDIDATE_NAME_" + str(i+1))
    cols.append("VOTES_WON_" + str(i+1))
    cols.append("CANDIDATE_PARTY_" + str(i+1))

### adds column to list
cols.append("TOTAL_VOTES_TALLIED")

### loop runs through all the candidates
for i in range(0, candidate_len):
    ### generates column name for each candidate and adds to list
    cols.append("VOTE_SHARE_" + str(i+1))

### adds columns to list
cols.append("LEAD_PARTY")
cols.append("LEAD_CANDIDATE")
cols.append("RAW_MARGIN")
cols.append("MARGIN_PERCENTPOINTS")
cols.append("CT_TOTAL_VOTES_TALLIED")

### loop runs through all the candidates
for i in range(0, candidate_len):
    ### generates column names for each candidate and adds to list
    cols.append("CT_VOTE_SHARE_" + str(i+1))

cols.append("CT_LEAD_PARTY")
cols.append("CT_LEAD_CANDIDATE")
cols.append("CT_RAW_MARGIN")
cols.append("CT_MARGIN_PERCENTPOINTS")
cols.append("CT_LEAD_VOTESHARE")
cols.append("CT_SECOND_PARTY")
cols.append("CT_SECOND_CANDIDATE")
cols.append("CT_SECOND_VOTESHARE")

### replaces column indices with generated column names
df.columns = cols

### displays dataframe
df

,LAST_UPDATE_2022,BASE_PRECINCT,CANDIDATE_NAME_1,VOTES_WON_1,CANDIDATE_PARTY_1,CANDIDATE_NAME_2,VOTES_WON_2,CANDIDATE_PARTY_2,CANDIDATE_NAME_3,VOTES_WON_3,...,CT_VOTE_SHARE_5,CT_VOTE_SHARE_6,CT_LEAD_PARTY,CT_LEAD_CANDIDATE,CT_RAW_MARGIN,CT_MARGIN_PERCENTPOINTS,CT_LEAD_VOTESHARE,CT_SECOND_PARTY,CT_SECOND_CANDIDATE,CT_SECOND_VOTESHARE
0,Last updated at 02-14-2023 03:16,1,Dennis Misigoy (LPF),1,LPF,Marco Rubio (REP),299,REP,Steven B. Grant (NPA),1,...,44.8,0.04,REP,Marco Rubio (REP),67273,9.45,54.25,DEM,Val Demings (DEM),44.8
1,Last updated at 02-14-2023 03:16,2,Dennis Misigoy (LPF),3,LPF,Marco Rubio (REP),594,REP,Steven B. Grant (NPA),6,...,44.8,0.04,REP,Marco Rubio (REP),67273,9.45,54.25,DEM,Val Demings (DEM),44.8
2,Last updated at 02-14-2023 03:16,3,Dennis Misigoy (LPF),5,LPF,Marco Rubio (REP),1024,REP,Steven B. Grant (NPA),11,...,44.8,0.04,REP,Marco Rubio (REP),67273,9.45,54.25,DEM,Val Demings (DEM),44.8
3,Last updated at 02-14-2023 03:16,4,Dennis Misigoy (LPF),10,LPF,Marco Rubio (REP),679,REP,Steven B. Grant (NPA),5,...,44.8,0.04,REP,Marco Rubio (REP),67273,9.45,54.25,DEM,Val Demings (DEM),44.8
4,Last updated at 02-14-2023 03:16,5,Dennis Misigoy (LPF),6,LPF,Marco Rubio (REP),1063,REP,Steven B. Grant (NPA),8,...,44.8,0.04,REP,Marco Rubio (REP),67273,9.45,54.25,DEM,Val Demings (DEM),44.8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
778,Last updated at 02-14-2023 03:17,994,Dennis Misigoy (LPF),4,LPF,Marco Rubio (REP),289,REP,Steven B. Grant (NPA),0,...,44.8,0.04,REP,Marco Rubio (REP),67273,9.45,54.25,DEM,Val Demings (DEM),44.8
779,Last updated at 02-14-2023 03:17,995,Dennis Misigoy (LPF),6,LPF,Marco Rubio (REP),968,REP,Steven B. Grant (NPA),3,...,44.8,0.04,REP,Marco Rubio (REP),67273,9.45,54.25,DEM,Val Demings (DEM),44.8
780,Last updated at 02-14-2023 03:17,996,Dennis Misigoy (LPF),2,LPF,Marco Rubio (REP),456,REP,Steven B. Grant (NPA),1,...,44.8,0.04,REP,Marco Rubio (REP),67273,9.45,54.25,DEM,Val Demings (DEM),44.8
781,Last updated at 02-14-2023 03:17,997,Dennis Misigoy (LPF),1,LPF,Marco Rubio (REP),135,REP,Steven B. Grant (NPA),0,...,44.8,0.04,REP,Marco Rubio (REP),67273,9.45,54.25,DEM,Val Demings (DEM),44.8


### Merging final dataframe with dataset of precinct demographics and prior results

In [15]:
### importing dataset with demographics and prior results
df_prior = pd.read_csv("dade_sen_prior.csv", keep_default_na = False)

### merging 
df_master = pd.merge(df, df_prior, on = "BASE_PRECINCT")

df_master["BASE_PRECINCT"] = df_master["BASE_PRECINCT"].apply(lambda x: "{0:0>3}".format(x))
#df_master["BASE_PRECINCT"] = str(df_master["BASE_PRECINCT"])

### loop runs through dataframe
for i in range(0, len(df_master)):
    ### converts the precinct numbers to string
    df_master["BASE_PRECINCT"][i] = str(df_master["BASE_PRECINCT"][i])
    ### converts the precincts to "000" format
    #df_master["BASE_PRECINCT"][i] = df_master["BASE_PRECINCT"][i].zfill(3)
    
### stores time
dt_string = tnow.strftime("%m-%d-%Y-%H%M")
### constructs filename
report = "sen_dade_report_" + str(dt_string) + ".csv"
path = "senate_dade/report_files/" + report

### exports dataframe to "governor_dade/report_files/"
df_master.to_csv(path, index = False)

### displays dataframe
df_master

,LAST_UPDATE_2022,BASE_PRECINCT,CANDIDATE_NAME_1,VOTES_WON_1,CANDIDATE_PARTY_1,CANDIDATE_NAME_2,VOTES_WON_2,CANDIDATE_PARTY_2,CANDIDATE_NAME_3,VOTES_WON_3,...,TOTAL_POP,RACE,PARTISANSHIP,DEMOCRAT_SHARE,REPUBLICAN_SHARE,WHITE_SHARE,BLACK_SHARE,HISPANIC_SHARE,OTHER_SHARE,MUNICIPALITY
0,Last updated at 02-14-2023 03:16,001,Dennis Misigoy (LPF),1,LPF,Marco Rubio (REP),299,REP,Steven B. Grant (NPA),1,...,762,Majority White,Other,26.4,32.5,65,0.3,25.9,8.9,Golden Beach
1,Last updated at 02-14-2023 03:16,002,Dennis Misigoy (LPF),3,LPF,Marco Rubio (REP),594,REP,Steven B. Grant (NPA),6,...,1810,Majority White,Other,25.6,31.5,51.5,1.3,35.7,11.4,Sunny Isles Beach
2,Last updated at 02-14-2023 03:16,003,Dennis Misigoy (LPF),5,LPF,Marco Rubio (REP),1024,REP,Steven B. Grant (NPA),11,...,2689,Majority White,Other,25.5,34.5,67.1,1.2,22.6,9.1,Sunny Isles Beach
3,Last updated at 02-14-2023 03:16,004,Dennis Misigoy (LPF),10,LPF,Marco Rubio (REP),679,REP,Steven B. Grant (NPA),5,...,2753,Other,Other,26.6,29,44.3,5,39.4,11.3,Sunny Isles Beach
4,Last updated at 02-14-2023 03:16,005,Dennis Misigoy (LPF),6,LPF,Marco Rubio (REP),1063,REP,Steven B. Grant (NPA),8,...,3427,Majority White,Other,24.2,31.5,54.8,2.6,31.7,10.9,Sunny Isles Beach
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
778,Last updated at 02-14-2023 03:17,994,Dennis Misigoy (LPF),4,LPF,Marco Rubio (REP),289,REP,Steven B. Grant (NPA),0,...,856,Majority Hispanic,Other,26.1,39.3,12.6,0.7,79.9,6.8,Miami
779,Last updated at 02-14-2023 03:17,995,Dennis Misigoy (LPF),6,LPF,Marco Rubio (REP),968,REP,Steven B. Grant (NPA),3,...,4980,Other,Other,30.5,25.3,46.7,3.8,37.3,12.1,Miami
780,Last updated at 02-14-2023 03:17,996,Dennis Misigoy (LPF),2,LPF,Marco Rubio (REP),456,REP,Steven B. Grant (NPA),1,...,3082,Other,Other,34.3,21.7,48.4,4.8,32.7,14,Miami
781,Last updated at 02-14-2023 03:17,997,Dennis Misigoy (LPF),1,LPF,Marco Rubio (REP),135,REP,Steven B. Grant (NPA),0,...,481,Majority Hispanic,Other,30.6,32.4,2.7,0.8,89.2,7.3,Miami
